In [39]:
import pandas as pd
import numpy as np
import importlib as imp
import os
import sys

sys.path.append("..")

from utils import kiwi_operators as kwo
from utils import config
from glob import glob
import matplotlib.pyplot as plt


# read in data and transform idx from datetime to int
item_lst = ["open", "high", "low", "close", "volume"]
dt_data = dict()
for item in item_lst:
    cur_data = pd.read_hdf(f"{config.dir_raw_data_min}/{item}.h5")
    cur_data = kwo.transform_minute_index_datetime_to_int(cur_data)
    dt_data[item] = cur_data
print("data loaded")


# post-reinstatement
df_gap = pd.read_hdf(f"{config.dir_raw_data_min}/pricegap.h5").fillna(0)
df_gap = kwo.transform_minute_index_datetime_to_int(df_gap)

for item in ["open", "high", "low", "close"]:
    df_adj = df_gap / dt_data[item] + 1
    df_adj = df_adj.reindex(dt_data[item].index)
    df_adj = df_adj.fillna(1)
    df_adj = df_adj.cumprod()
    for col in df_adj.columns:
        df_adj[col] = df_adj[col] / df_adj[col].iloc[-1]
    dt_data[item] = dt_data[item] * df_adj
print("reinstatement finished")

# get amount
df_size = pd.read_pickle(f"{config.dir_data_day}/Size_MainAdj.pkl")
close_dates = pd.Series(dt_data["close"].index // 10000).unique()
df_size = df_size.reindex(index=close_dates).fillna(method="ffill")
df_size.index = df_size.index * 10000 + 900
close_idx = set(dt_data["close"].index)
size_idx = set(df_size.index)
union_idx = close_idx.union(size_idx)
union_idx = sorted(list(union_idx))
df_size = df_size.reindex(union_idx).fillna(method="ffill")
df_size = df_size.reindex(dt_data["close"].index)
df_amount = df_size * dt_data["close"] * dt_data["volume"]
dt_data["amount"] = df_amount
print("amount generated")

# save data
for item in dt_data.keys():
    dt_data[item].to_pickle(f"{config.dir_data_min}/{item}.pkl")
print("data saved")


data loaded
reinstatement finished
amount generated
data saved
